In [1]:
from pyspark import SparkContext, SparkConf
conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

In [2]:
df_users = sc.textFile('./moviedata/users.csv')
df_zipcodes = sc.textFile('./moviedata/zipcodes.csv')
df_rating = sc.textFile('./moviedata/rating.csv')
df_movies = sc.textFile('./moviedata/movies.csv')

In [3]:
zipcode_user = df_users.map(lambda x: x.split(",")).map(lambda x:  (str(x[4])[1:-1],str(x[0])))

In [4]:
zipcode_user.take(5)

[('94560', '780'),
 ('48825', '781'),
 ('77081', '783'),
 ('91040', '784'),
 ('23322', '785')]

In [5]:
zipcode_state = df_zipcodes.map(lambda x: x.split(",")).map(lambda x:  (str(x[0])[1:-1], str(x[3])[1:-1]))

In [6]:
zipcode_state.take(5)

[('2574', 'MA'),
 ('1886', 'MA'),
 ('1472', 'MA'),
 ('2671', 'MA'),
 ('2672', 'MA')]

In [7]:
zipcode_user_state = zipcode_user.join(zipcode_state)

In [8]:
zipcode_user_state.take(5)

[('10707', ('95', 'NY')),
 ('98101', ('6', 'WA')),
 ('11753', ('924', 'NY')),
 ('6811', ('403', 'CT')),
 ('37076', ('874', 'TN'))]

In [9]:
user_state = zipcode_user_state.map(lambda x: x[1])

In [10]:
user_state.take(5)

[('95', 'NY'), ('6', 'WA'), ('924', 'NY'), ('403', 'CT'), ('874', 'TN')]

In [11]:
user_movieid_rating = df_rating.map(lambda x: x.split(",")).map(lambda x: (str(x[0]), (str(x[1]),int(x[2]))))

In [12]:
user_movieid_rating.take(5)

[('253', ('97', 4)),
 ('284', ('269', 4)),
 ('106', ('526', 4)),
 ('121', ('180', 3)),
 ('62', ('86', 2))]

In [13]:
user_movieid_rating_state = user_movieid_rating.join(user_state)

In [14]:
user_movieid_rating_state.take(5)

[('214', (('117', 4), 'NY')),
 ('214', (('137', 4), 'NY')),
 ('214', (('213', 4), 'NY')),
 ('214', (('302', 4), 'NY')),
 ('214', (('693', 3), 'NY'))]

In [15]:
movieid_state_rating = user_movieid_rating_state.map(lambda x: (x[1][0][0], (x[1][1], x[1][0][1])))

In [16]:
movieid_state_rating.take(5)

[('117', ('NY', 4)),
 ('137', ('NY', 4)),
 ('213', ('NY', 4)),
 ('302', ('NY', 4)),
 ('693', ('NY', 3))]

In [17]:
def clean_df_movies(x):
    y = x.split('\"')
    return y[0].split(',')[0:1] + y[2].split(',')[2:]

In [18]:
df_movies_cleaned = df_movies.map(lambda x : clean_df_movies(x))

In [19]:
for i in range(3, 20):
    movieid_list = df_movies_cleaned.filter(lambda x: int(x[i]) == 1).map(lambda x: (str(x[0]), i))
    state_rating = movieid_list.join(movieid_state_rating).map(lambda x: (x[1][1][0], (x[1][1][1], 1)))
    state_avg_rating = state_rating.reduceByKey(lambda x,y : (x[0] + y[0], x[1]+y[1])).map(lambda x : (x[0], float(x[1][0])/x[1][1]))
    print state_avg_rating.takeOrdered(1, key = lambda x: -x[1])

[('ME', 4.5)]
[('MS', 5.0)]
[('MS', 4.444444444444445)]
[('WV', 4.666666666666667)]
[('WV', 4.833333333333333)]
[('SD', 5.0)]
[('WV', 4.767441860465116)]
[('AL', 5.0)]
[('HI', 5.0)]
[('ME', 5.0)]
[('ME', 4.375)]
[('WV', 4.666666666666667)]
[('WV', 4.5)]
[('WV', 4.476190476190476)]
[('WV', 4.5)]
[('WV', 4.684210526315789)]
[('ME', 4.333333333333333)]
